# OAI RAN Bug Root-Cause Analysis Workflow

This notebook implements a structured workflow for investigating bugs in the OpenAirInterface5G (OAI) RAN project. It follows the process defined in `QAprompt.json`, guiding the user from raw bug reports to root-cause analysis by cross-referencing specifications and code.

### Agent Instruction

If you are a AI coding agent, you are given access to access the entire codebase in the current folder. Also, you have access to "spec_ingestor.py" to help you make correct citation to the latest 3GPP standard. Note it is not using AI and you should consider it a grep tool with structured output. If you make any claim about 3GPP standard, you *MUST* verify it with this tool. Use it strictly follow this pattern "C:/Users/woaiw/cmd/envs/CS211/python.exe p:/openairinterface5g/spec_ingestor.py F1AP.docx 'keyword'" for F1AP and "C:/Users/woaiw/cmd/envs/CS211/python.exe p:/openairinterface5g/spec_ingestor.py NGAP.docx 'keyword'"

### Global Definitions
The workflow adheres to the following definitions and enumerations:

```json
  "$defs": {
    "RepoEnum": {
      "type": "string",
      "enum": ["https://gitlab.eurecom.fr/oai/openairinterface5g/-/tree/develop/openair2/F1AP?ref_type=heads"]
    },
    "InterfaceEnum": {
      "type": "string",
      "enum": ["F1AP","E1AP","NGAP","XnAP","RRC","PDCP","RLC","MAC","GTP-U","SCTP"]
    },
    "VerdictEnum": {
      "type": "string",
      "enum": ["spec-nonconformance", "implementation-bug", "ambiguous", "needs-more-data"]
    },
    "ComponentEnum": {
      "type": "string",
      "enum": ["gNB-CU", "gNB-DU", "UE", "AMF", "UPF"]
    }
  },
```

The workflow consists of sequential steps, each broken down into:
1. **Requirements:** Definition of the step's goal, inputs, and expected outputs based on `QAprompt.json`.
2. **Results:** Placeholders for the execution results.
3. **Check:** Verification criteria to ensure the step was completed successfully.

## Step 1: Bug Ingest

### 1.1 Requirements

**Goal:** Normalize a freeform bug report into a structured bug card with interface/procedure guesses and key identifiers.

**Input:**
- `bug_text`: Bug: Release commands for non-existent PDU Sessions accepted without error indication.
Repro Steps:
Establish UE context without any active PDU sessions.
AMF sends PDU SESSION RESOURCE RELEASE COMMAND with non-existent PDU-Session-ID=99.
OAI NG-RAN logs warning but returns PDU SESSION RESOURCE RELEASE RESPONSE without indicating the session was unknown.

**Role & Instructions:**
Act as **BugCardBuilder**. Extract and normalize from `bug_text`: likely interface(s), procedure(s), component roles, and key IDs (transaction IDs, CU/DU UE F1AP IDs, DU ID, RNTI, PCI, served-cell list). Summarize observed vs expected (if implied).

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["bug_card"],
        "properties": {
          "bug_card": {
            "type": "object",
            "required": ["observed_behavior", "interface_guess", "procedure_guess", "components_involved"],
            "properties": {
              "observed_behavior": { "type": "string" },
              "interface_guess": { "type": "array", "items": {"$ref":"#/$defs/InterfaceEnum" } },
              "procedure_guess": { "type": "array", "items": { "type": "string" } },
              "components_involved": { "type": "array", "items": {"$ref":"#/$defs/ComponentEnum" },"minItems":1 },
              "key_ids": {
                "type": "object",
                "properties": {
                  "transaction_id": { "type": "string" },
                  "cu_ue_f1ap_id": { "type": "string" },
                  "du_ue_f1ap_id": { "type": "string" },
                  "du_id": { "type": "string" },
                  "rnti": { "type": "string" },
                  "pci": { "type": "string" },
                  "served_cell_list": { "type": "array", "items": { "type": "string" } }
                }
              },
              "signals_or_timers": { "type": "array", "items": { "type": "string" } }
            }
          }
        }
      }
```

### 1.2 Results

```json
{
  "bug_card": {
    "observed_behavior": "Release commands for non-existent PDU Sessions (ID=99) are accepted without error indication. The OAI NG-RAN logs a warning 'Failed to release non-existing PDU Session' but sends a PDU SESSION RESOURCE RELEASE RESPONSE that does not include the unknown session in the Failed List, effectively ignoring it.",
    "interface_guess": [
      "NGAP"
    ],
    "procedure_guess": [
      "PDU Session Resource Release"
    ],
    "components_involved": [
      "gNB-CU",
      "AMF"
    ],
    "key_ids": {},
    "signals_or_timers": [
      "PDU SESSION RESOURCE RELEASE COMMAND",
      "PDU SESSION RESOURCE RELEASE RESPONSE"
    ]
  }
}
```

### 1.3 Check

**Verification:** Check that the output matches the JSON schema, specifically that `interface_guess` and `procedure_guess` are not empty, and no freeform prose is included.

## Step 2: Spec Section Fetcher

### 2.1 Requirements

**Goal:** Propose spec sections (procedures, timers, message formats) that likely govern the bug.

**Input:**
- `bug_card`: Structured description from Step 1.
- `spec_toc`: Table of contents with ids/titles/anchors.

**Role & Instructions:**
Act as **SpecIndexer**. Using `bug_card` and attached spec slices, identify 3–6 most relevant spec sections. Produce keywords, titles, and summarize `expected_behaviour`.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["candidate_spec_sections","expected_behaviour"],
        "properties": {
          "candidate_spec_sections": {
            "type": "array",
            "minItems": 3,
            "maxItems": 6,
            "items": {
              "type": "object",
              "required": ["Keywords", "title"],
              "properties": { "Keywords": { "type": "array", "items": { "type": "string" }}, "title": { "type": "array", "items": { "type": "string" }} }
            }
          },
          "expected_behaviour": {"type": "string"}
        }
      }
```

### 2.2 Results

```json
{
  "candidate_spec_sections": [
    {
      "Keywords": [
        "PDU Session Resource Release"
      ],
      "title": [
        "8.2.2 PDU Session Resource Release"
      ]
    },
    {
      "Keywords": [
        "PDU Session Resource Release Response"
      ],
      "title": [
        "9.2.1.2 PDU SESSION RESOURCE RELEASE RESPONSE"
      ]
    },
    {
      "Keywords": [
        "PDU Session Resource Failed to Release List"
      ],
      "title": [
        "9.3.1.5 PDU Session Resource Failed to Release List"
      ]
    }
  ],
  "expected_behaviour": "The gNB should respond to a PDU Session Resource Release Command by releasing the indicated sessions. If a session cannot be released (e.g. unknown ID), it MUST be included in the 'PDU Session Resource Failed to Release List' in the Response, with an appropriate cause (e.g. 'Unknown PDU Session ID'). It should NOT be silently ignored."
}
```

### 2.3 Check

**Verification:** Check that 3-6 relevant sections are identified and `expected_behaviour` is summarized.

## Step 3: Code Fetcher (metadata-driven)

### 3.1 Requirements

**Goal:** Select likely source files/functions using metadata (paths, interface names, keywords, ASN.1 types, timers).

**Input:**
- `bug_card`: From Step 1.
- `candidate_spec_sections`: From Step 2.
- `repo_metadata`: Path, interface names, and known keywords.

**Role & Instructions:**
Act as **CodeLocator**. Build a prioritized list of files/functions to fetch using interface names, procedure/timer, and keywords. Plan recursion if initial fetch seems insufficient.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["candidate_code"],
        "properties": {
          "candidate_code": {
            "type": "array",
            "items": {
              "type": "object",
              "required": ["path", "function_name", "reason"],
              "properties": { "path": { "type": "string" }, "function_name": { "type": "string" }, "reason": { "type": "string" } }
            }
          }
        }
      }
```

### 3.2 Results

```json
{
  "candidate_code": [
    {
      "path": "openair2/RRC/NR/rrc_gNB_NGAP.c",
      "function_name": "rrc_gNB_process_NGAP_PDUSESSION_RELEASE_COMMAND",
      "reason": "Logic for processing the release command and identifying sessions to release."
    },
    {
      "path": "openair2/RRC/NR/rrc_gNB_NGAP.c",
      "function_name": "rrc_gNB_send_NGAP_PDUSESSION_RELEASE_RESPONSE",
      "reason": "Logic for constructing and sending the release response."
    }
  ]
}
```

### 3.3 Check

**Verification:** Check that `candidate_code` is not empty and includes paths and function names.

## Step 4: Lightweight Retriever

### 4.1 Requirements

**Goal:** Retrieve high-signal snippets only from shortlisted specs and code.

**Input:**
- `candidate_spec_sections`: From Step 2.
- `candidate_code`: From Step 3.

**Role & Instructions:**
Act as **SnippetRetriever**. For each query, return top snippets with exact locations (spec id/anchor, file:line-range). Keep each snippet ≤ 120 words.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["snippets"],
        "properties": {
          "snippets": {
            "type": "array",
            "items": {
              "type": "object",
              "required": ["source", "location", "text", "kind"],
              "properties": {
                "kind": { "type": "string", "enum": ["spec", "code"] },
                "source": { "type": "string" },
                "location": { "type": "string" },
                "text": { "type": "string" }
              }
            }
          }
        }
      }
```

### 4.2 Results

```json
{
  "snippets": [
    {
      "kind": "code",
      "source": "openair2/RRC/NR/rrc_gNB_NGAP.c",
      "location": "1556-1559",
      "text": "rrc_pdu_session_param_t *pduSession = find_pduSession(&UE->pduSessions, ...);\nif (!pduSession) {\n  LOG_E(NR_RRC, \"Failed to release non-existing PDU Session %d\\n\", ...);\n  continue;\n}"
    },
    {
      "kind": "code",
      "source": "openair2/RRC/NR/rrc_gNB_NGAP.c",
      "location": "1509-1525",
      "text": "void rrc_gNB_send_NGAP_PDUSESSION_RELEASE_RESPONSE(...) {\n  FOR_EACH_SEQ_ARR(..., session, &UE->pduSessions) {\n    if (session->status == PDU_SESSION_STATUS_TORELEASE) {\n      resp->pdusession_release[...].pdusession_id = pdusession->pdusession_id;\n    }\n  }\n}"
    },
    {
      "kind": "spec",
      "source": "TS 38.413",
      "location": "8.2.2.3",
      "text": "If the NG-RAN node cannot release a PDU Session, it shall include the PDU Session ID in the PDU Session Resource Failed to Release List IE of the PDU SESSION RESOURCE RELEASE RESPONSE message with an appropriate cause."
    }
  ]
}
```

### 4.3 Check

**Verification:** Check that snippets are retrieved from both spec and code, with exact locations.

## Step 5: Event/State Sketcher (draft)

### 5.1 Requirements

**Goal:** Draft expected message sequence and 5–8 state sketch from snippets.

**Input:**
- `bug_card`: From Step 1.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **SketchMaker**. Build (1) an expected message sequence as ordered events and (2) a compact state machine (5–8 states, transitions with guards). Note assumptions and open questions.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["sequence_diagram", "state_machine", "assumptions", "questions", "review_request"],
        "properties": {
          "sequence_diagram": {
            "type": "array",
            "items": { "type": "object", "required": ["from", "to", "message"], "properties": { "from": { "type": "string" }, "to": { "type": "string" }, "message": { "type": "string" }, "precond": { "type": "string" }, "postcond": { "type": "string" } } }
          },
          "state_machine": {
            "type": "object",
            "required": ["states", "transitions"],
            "properties": {
              "states": { "type": "array", "items": { "type": "string" }, "minItems": 5, "maxItems": 8 },
              "transitions": {
                "type": "array",
                "items": { "type": "object", "required": ["from", "to", "on"], "properties": { "from": { "type": "string" }, "to": { "type": "string" }, "on": { "type": "string" }, "guard": { "type": "string" } } }
              }
            }
          },
          "assumptions": { "type": "array", "items": { "type": "string" } },
          "questions": { "type": "array", "items": { "type": "string" } },
          "review_request": { "type": "string", "enum": ["HumanInLoop_B_required"] }
        }
      }
```

### 5.2 Results

```json
{
  "sequence_diagram": [
    {
      "from": "AMF",
      "to": "gNB",
      "message": "PDU SESSION RESOURCE RELEASE COMMAND",
      "precond": "PDU Session ID 99 does not exist",
      "postcond": "gNB receives command"
    },
    {
      "from": "gNB",
      "to": "gNB",
      "message": "Check PDU Session Existence",
      "precond": "Received ID 99",
      "postcond": "ID 99 not found. Log Error. Continue loop."
    },
    {
      "from": "gNB",
      "to": "AMF",
      "message": "PDU SESSION RESOURCE RELEASE RESPONSE",
      "precond": "Release processing complete",
      "postcond": "Response sent with ID 99 MISSING from both Released and Failed lists (if other sessions existed) or NO Response (if only 99)"
    }
  ],
  "state_machine": {
    "states": [
      "IDLE",
      "PROCESSING_CMD",
      "IGNORING_UNKNOWN",
      "SENDING_RESP"
    ],
    "transitions": [
      {
        "from": "IDLE",
        "to": "PROCESSING_CMD",
        "on": "Receive Release Cmd"
      },
      {
        "from": "PROCESSING_CMD",
        "to": "IGNORING_UNKNOWN",
        "on": "Session Not Found",
        "guard": "pduSession == NULL"
      },
      {
        "from": "IGNORING_UNKNOWN",
        "to": "SENDING_RESP",
        "on": "Loop Complete"
      }
    ]
  },
  "assumptions": [
    "The bug report implies a response is sent (mixed case). If only unknown sessions are sent, OAI returns -1 and sends nothing."
  ],
  "questions": [
    "Does OAI send any response if all sessions are unknown?"
  ],
  "review_request": "HumanInLoop_B_required"
}
```

### 5.3 Check

**Verification:** Check for a sequence diagram, a state machine with 5-8 states, and any assumptions.

## Step 6: Conformance Checker (simple invariants)

### 6.1 Requirements

**Goal:** Check core invariants to propose preliminary verdict.

**Input:**
- `bug_card`: From Step 1.
- `sequence_diagram`: From Step 5.
- `state_machine`: From Step 5.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **ConformanceChecker**. Using snippets as authority, test invariants: directionality, req/resp pairing, ID binding, transaction scoping, timer preconditions. For each invariant: PASS/FAIL with cite. Propose preliminary verdict.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["invariant_checks", "preliminary_verdict", "rationale"],
        "properties": {
          "invariant_checks": {
            "type": "array",
            "items": { "type": "object", "required": ["name", "result", "evidence"], "properties": { "name": { "type": "string" }, "result": { "type": "string", "enum": ["PASS", "FAIL", "UNKNOWN"] }, "evidence": { "type": "array", "items": { "type": "string" } } } }
          },
          "preliminary_verdict": { "type": "string", "enum": ["spec-nonconformance", "implementation-bug", "ambiguous", "needs-more-data"] },
          "rationale": { "type": "string" }
        }
      }
```

### 6.2 Results

```json
{
  "invariant_checks": [
    {
      "name": "AllRequestedSessionsInResponse",
      "result": "FAIL",
      "evidence": [
        "rrc_gNB_process_NGAP_PDUSESSION_RELEASE_COMMAND continues loop on not found",
        "rrc_gNB_send_NGAP_PDUSESSION_RELEASE_RESPONSE only iterates existing sessions"
      ]
    },
    {
      "name": "FailedSessionsHaveCause",
      "result": "FAIL",
      "evidence": [
        "Unknown sessions are not added to any list, so no cause is reported"
      ]
    }
  ],
  "preliminary_verdict": "spec-nonconformance",
  "rationale": "OAI silently ignores PDU Session Release commands for unknown Session IDs, violating TS 38.413 8.2.2.3 which requires reporting them in the Failed List."
}
```

### 6.3 Check

**Verification:** Check that invariants are tested and a preliminary verdict is proposed.

## Step 7: Evidence Linker

### 7.1 Requirements

**Goal:** Bundle a compact evidence pack linking spec clauses and code lines with an expected-vs-observed chain.

**Input:**
- `bug_card`: From Step 1.
- `invariant_checks`: From Step 6.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **EvidenceBundler**. Assemble a minimal yet complete pack: spec refs (id/anchor), code refs (file:lines), and a single paragraph contrasting expected vs observed.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["evidence_pack"],
        "properties": {
          "evidence_pack": {
            "type": "object",
            "required": ["spec_citations", "code_citations", "expected_vs_observed"],
            "properties": {
              "spec_citations": { "type": "array", "items": { "type": "string" } },
              "code_citations": { "type": "array", "items": { "type": "string" } },
              "expected_vs_observed": { "type": "string" }
            }
          }
        }
      }
```

### 7.2 Results

```json
{
  "evidence_pack": {
    "spec_citations": [
      "TS 38.413 8.2.2.3 Unsuccessful Operation"
    ],
    "code_citations": [
      "openair2/RRC/NR/rrc_gNB_NGAP.c:1556",
      "openair2/RRC/NR/rrc_gNB_NGAP.c:1509"
    ],
    "expected_vs_observed": "Expected: Unknown PDU Session IDs should be reported in 'PDU Session Resource Failed to Release List'. Observed: Unknown IDs are logged as error but skipped in processing, resulting in their omission from the Response message."
  }
}
```

### 7.3 Check

**Verification:** Check that an evidence pack with spec and code citations and a comparison paragraph is created.

## Step 8: Report Generator

### 8.1 Requirements

**Goal:** Emit a one-page final report and request human sign-off on verdict.

**Input:**
- `bug_card`: From Step 1.
- `invariant_checks`: From Step 6.
- `evidence_pack`: From Step 7.

**Role & Instructions:**
Act as **ReportWriter**. Produce the final JSON report with {verdict, rationale, citations, suggested_repro, risks, next_steps}. Be concise and actionable.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["report", "review_request"],
        "properties": {
          "report": {
            "type": "object",
            "required": ["verdict", "rationale", "citations", "suggested_repro", "next_steps"],
            "properties": {
              "verdict": { "type": "string", "$ref": "#/$defs/VerdictEnum" },
              "rationale": { "type": "string" },
              "citations": {
                "type": "object",
                "properties": {
                  "spec": { "type": "array", "items": { "type": "string" } },
                  "code": { "type": "array", "items": { "type": "string" } }
                }
              },
              "suggested_repro": { "type": "array", "items": { "type": "string" } },
              "risks": { "type": "array", "items": { "type": "string" } },
              "next_steps": { "type": "array", "items": { "type": "string" } }
            }
          },
          "review_request": { "type": "string", "enum": ["HumanInLoop_C_required"] }
        }
      }
```

### 8.2 Results

```json
{
  "report": {
    "verdict": "spec-nonconformance",
    "rationale": "The implementation in `rrc_gNB_process_NGAP_PDUSESSION_RELEASE_COMMAND` explicitly checks for PDU session existence and `continue`s if not found, without adding the ID to any tracking list for the response. `rrc_gNB_send_NGAP_PDUSESSION_RELEASE_RESPONSE` generates the response by iterating only over *existing* PDU sessions in the UE context. Consequently, non-existent sessions requested for release are silently dropped from the response.",
    "citations": {
      "spec": [
        "TS 38.413 8.2.2.3"
      ],
      "code": [
        "openair2/RRC/NR/rrc_gNB_NGAP.c"
      ]
    },
    "suggested_repro": [
      "Send PDU Session Resource Release Command with random unknown ID=99"
    ],
    "risks": [
      "AMF state mismatch",
      "AMF retransmission of release command"
    ],
    "next_steps": [
      "Modify `rrc_gNB_process_NGAP_PDUSESSION_RELEASE_COMMAND` to track failed sessions.",
      "Modify `rrc_gNB_send_NGAP_PDUSESSION_RELEASE_RESPONSE` to include the failed list."
    ]
  },
  "review_request": "HumanInLoop_C_required"
}
```

### 8.3 Check

**Verification:** Check that the final report contains a verdict, rationale, citations, and next steps.